# Generate figures for fMRI report

In [ ]:
# Load necessary items

%pylab inline
np.set_printoptions(precision=2, suppress=True) # Set numpy to print only 2 decimal digits for neatness

import warnings
warnings.filterwarnings('ignore')

import math
import ast

#import pylab as plt
import nibabel as nb

import os, os.path
import json
from os.path import join as opj
import pprint

from scipy.io.matlab import loadmat

from nilearn import image as nli
from nilearn.plotting import plot_stat_map
from nilearn.plotting import plot_epi
from nilearn.plotting import plot_anat
from nilearn.plotting import plot_glass_brain
from nilearn.plotting import plot_stat_map
from nilearn.image import resample_to_img
from nilearn.image import math_img

import pandas as pd

from IPython.display import SVG, Image, Markdown, HTML, clear_output

import shutil

import csv
import operator

# First level
import pandas as pd
from nipype.interfaces.base import Bunch

import sys
from PIL import Image as pimage
from IPython.display import Image

from cairosvg import svg2png

In [ ]:
# Define some parameters for the pipeline and the functions

wDir = '/data/Patients'
infoDir = '/data/Patients/SubjectInfoFiles'
oDir = '/data/output'
iDir = '/data/Patients/SubjectInfoFiles'
patDcmRoot = '/data/Patients/tmp_dicom'
bidsSource = '/data/BIDS_source'

# For the LI curves:
aalNii = '/opt/spm12/toolbox/aal_for_SPM12/atlas/AAL.nii'
aalLabels = '/data/masks/ROI_MNI_V4.csv'
aalImg = nb.load(aalNii)
aalROIs = pd.read_csv(aalLabels, delimiter='\t')

brainRegions = ['Precentral', 'Frontal', 'Rolandic', 'Supp_Motor', 'Olfactory', 'Rectus', 'Insula', 'Cingulum', 
           'Hippocampus', 'ParaHippocampal', 'Amygdala', 'Calcarine', 'Cuneus', 'Lingual', 'Occipital', 'Fusiform', 
           'Postcentral', 'Parietal', 'SupraMarginal', 'Angular', 'Precuneus', 'Paracentral_Lobule', 'Caudate', 'Putamen',
           'Pallidum', 'Thalamus', 'Heschl', 'Temporal', 'Cerebelum', 'Vermis']

enconewDirlist = []

## Define functions

In [ ]:
def pause():
    programPause = input()

In [ ]:
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    fontstr = "<font"
    display(Markdown(colorstr))

### Select subjects

In [ ]:
# Subject selection:
def subjects4report():
    """
    
    Screens all entrys with sub-<#> in wDir for completion of datatreatment
    
    returns: patientStatList = [subdir, [patientinfo = dcm tags], {figures info per task}]
        
    """
    
    # Here we define the list of subjects in wDir: subList
    subList = []
    for item in os.listdir(infoDir):
        if item.endswith('_log.json'):
            subList.append(item)
    
    # Create a list 'patientList' with for every patient the following elements:
    # [[ sub-<accession#>, {dcmtags dict}, [list of 1stLevel tasks]], [idem], ....]
    
    patientStatList = []
            
    for i, sub in enumerate(subList):
        subInfoList = []
        subID = sub.split('_')[0]
        jsonFile = opj(infoDir, sub)
        with open(jsonFile, "r") as jFile:
            jData = json.load(jFile)
        if jData["Statistics"] != ['None']:
            subInfoList.append(subID)
            subInfoList.append(jData["Subject DCM tags"])
            subInfoList.append(jData["Statistics"])
            patientStatList.append(subInfoList)

    return patientStatList

In [ ]:
# List subjects without report
def subjectsWithoutReport():
    """
    Generates a list of subjects without report as from the /data/Patients/SubjectInfoFiles/subjects.tsv
    
    """
    
    subjectsWOR = []
    
    with open(opj(wDir, 'SubjectInfoFiles', 'subjects.tsv'), 'rt') as tsv_file:
        reader = csv.reader(tsv_file, delimiter='\t')
        lines = list(reader)

    figDict = {}
    for line in lines:
        #print('%s -> %s' %(line[2], line[-2]))
        figDict[line[2]] = line[-2]

    item = 0
    for subject in sorted(lines):
        subIDnum = subject[2]
        if subject[-2] == 'N':
            item += 1
            print('%i: sub-%s - > %s, %s' % (item, subject[2], subject[0], subject[5]))
            subjectsWOR.append(subject)
    
    return subjectsWOR

In [ ]:
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

In [ ]:
def select_subject4figures(pList):
    """
    If no subject is selected, the procedure will be aborted
    A list of available subjects and their finished 1st-level tasks will be shown,
    
    : returns: subID and firstlevelList of selected
        
    """
    
    for i, subject in enumerate(subjectsWOR):
        subName = subjectsWOR[i][0]
        subDir = 'sub-' + subjectsWOR[i][2]
        subParadigms = subjectsWOR[i][5]
        print('%i = %s (%s) with %s' %(i+1, subName, subDir, subParadigms))
        
    subSelectNumber = int(eval(input('Select a subject to generate figures by entering the number: ')))
    
    subID = 'sub-' + pList[subSelectNumber-1][2]
    firstlevelList = ast.literal_eval(pList[subSelectNumber-1][5])
    
    #for i in range(len(firstlevelList)):
    #    firstlevelList[i] = str(firstlevelList[i])
    
    return subID, firstlevelList

### show_slices, plot_art

In [ ]:
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")

In [ ]:
def plot_art(task):
    """ 
    Function to plot the SVG file to screen
    subID, subDir, outputDir and subDatasinkDir are defined
    
    """
    
    fileID = subID + '_task-' + task + '_bold'
    fName = opj(subDatasinkDir, 'preproc', 'plot.r' + fileID + '.svg')
    if os.path.isfile(fName) is True:
        print('Artefact anaysis result for ' + task.upper())
        display(SVG(filename = fName))
    else:
        print(fName + " doesn't exist!")

### plot_detrended_voxel

In [ ]:
def plot_detrended_voxel():
    """
    Procedure to select a relevant voxel to plot trendy curves,
    followed by plot of curves in that voxel for all tasks
    
    subID, subDir, outputDir and subDatasinkDir are defined
        
    """
    
    task = firstlevelList[0]
    
    fileID = subID + '_task-' + task + '_bold'
    
    output = opj(outputDir, 'work_preproc_' + task)
    try:
        mc = nb.load(opj(output, 'applywarp_' + task, 'ra' + fileID + '_flirt.nii'))
    except:
        mc = nb.load(opj(output, 'applywarp_' + task, 'r' + fileID + '_flirt.nii'))
    
    if os.path.isdir(opj(output, 'susan_' + task)):
        smooth = nb.load(opj(output, 'susan_' + task,  'smooth', 'mapflow', '_smooth0', 'ra' + fileID + '_flirt_smooth.nii.gz'))
    elif os.path.isdir(opj(output, 'smooth_' + task)):
        try:
            smooth = nb.load(opj(output, 'smooth_' + task, 'sra' + fileID + '_flirt.nii'))
        except:
            smooth = nb.load(opj(output, 'smooth_' + task, 'sr' + fileID + '_flirt.nii'))
                  
    detrendedFile = nb.load(opj(subDatasinkDir, 'preproc', 'detrend-' + task + '.nii.gz'))
    detrended = detrendedFile.get_data()
    detrShape = detrended.shape
    midX = detrShape[0]//2
    midY = detrShape[1]//2
    midZ = detrShape[2]//2
        
    slice0 = detrended[midX, :, :, 1]
    slice1 = detrended[:, midY, :, 1]
    slice2 = detrended[:, :, midZ, 1]
    show_slices([slice0, slice1, slice2])
    plt.suptitle('Center slices for detrended image for paradigm ' + task)
    plt.pause(0.05)
        
    # try to select an axial slice in the brain for displaying
    zSlice = int(input('Select a z coordinate for displaying a brain slice:'))
        
    slice2 = detrended[:, :, zSlice, 1]
    show_slices([slice0, slice1, slice2])

    plt.suptitle("Selected axial slice.")
    plt.grid(True)
    plt.pause(0.05)
    
    x = int(input('Select a valid x coordinate for display of time signal in detrend.nii.gz:'))
    y = int(input('Select a valid y coordinate for display of time signal in detrend.nii.gz:'))
    z = zSlice
    
    for task in firstlevelList:
        output = opj(outputDir, 'work_preproc_' + task)
        fileID = subID + '_task-' + task + '_bold'
        try:
            mc = nb.load(opj(output, 'applywarp_' + task, 'ra' + fileID + '_flirt.nii'))
        except:
            mc = nb.load(opj(output, 'applywarp_' + task, 'r' + fileID + '_flirt.nii'))
                  
        if os.path.isdir(opj(output, 'susan_' + task)):
            smooth = nb.load(opj(output, 'susan_' + task,  'smooth', 'mapflow', '_smooth0', 'ra' + fileID + '_flirt_smooth.nii.gz'))
        elif os.path.isdir(opj(output, 'smooth_' + task)):
            try:
                smooth = nb.load(opj(output, 'smooth_' + task, 'sra' + fileID + '_flirt.nii'))
            except:
                smooth = nb.load(opj(output, 'smooth_' + task, 'sr' + fileID + '_flirt.nii'))
        
        detrendedFile = nb.load(opj(subDatasinkDir, 'preproc', 'detrend-' + task + '.nii.gz'))
        detrended = detrendedFile.get_data()
        fig = plt.figure(figsize=(12, 4))
        plt.suptitle('Timeline of voxel ' + '[' + str(x) + ',' + str(y) + ',' + str(z) +'] for ' + task.upper())
        plt.plot(mc.get_data()[x, y, z, :])
        plt.plot(smooth.get_data()[x, y, z, :])
        plt.plot(detrendedFile.get_data()[x, y, z, :])
        plt.legend(['motion_corrected', 'smoothed', 'detrended']);
        plt.savefig(opj(subDatasinkDir, 'figures', 'detrended_curve_' + task + '_loc_' + str(x) + '_' + str(y) + '_' + str(z) + '.png'))
        plt.pause(0.05)
    
    voxel = (x, y, z)
    
    return voxel

### file_len, select_task

In [ ]:
# for finding the number of outliers, we count the number of lines
def file_len(fName):
    with open(fName) as f:
        for i, l in enumerate(f):
            pass
    try:
        return i + 1
    except:
        return 0

In [ ]:
def select_task():
    """
    Function to select the task for reporting SPM results
    the variable 'firstlevelList' must be defined
    
    returns variable 'task'
    """
    
    print('Task(s) for evaluation: %s' % firstlevelList)
    
    task = input('Choose a task? ')
    if task in firstlevelList:
        print(task)
    else:
        print('Please select one in the list!')
        raise
        
    return task

### contrastList_gen

In [ ]:
def contrastList_gen():
    '''
    function to define a list with all the contrasts
    firstlevelList is defined
        
    '''

    contrastList = []
    subjectInfoList = []

    for i, task in enumerate(firstlevelList):
        eventFile = opj(subDir, 'func', subID + '_task-' + task + '_events.tsv')
        trialInfo = pd.read_table(eventFile)
        #trialInfo = pd.read_table(opj(bidsSource, task + '_seconds.tsv'))

        conditions = []
        onsets = []
        durations = []

        for group in trialInfo.groupby('trial_type'):
            conditions.append(group[0])
            onsets.append(list(group[1].onset))
            durations.append(group[1].duration.tolist())

        subjectInfo = [Bunch(conditions=conditions,
                          onsets=onsets,
                          durations=durations,
                          )]

        subjectInfoList.append('subject_info_' + task)
        subjectInfoList[i] = subjectInfo

        conditionNames = []
        if task == 'enco_new':
            conditionNames = [conditions[1], conditions[0]]
        else:
            conditionNames = [conditions[0], conditions[1]]

        # Contrasts
        cont01 = ['average', 'T', conditionNames, [1/2., 1/2.]]
        cont02 = [conditionNames[0], 'T', conditionNames, [1, 0]]
        cont03 = [conditionNames[1],         'T', conditionNames, [0, 1]]
        cont04 = [conditionNames[0] + ' >> ' + conditionNames[1], 'T', conditionNames, [1, -1]]
        cont05 = [conditionNames[1] + ' >> ' + conditionNames[0], 'T', conditionNames, [-1, 1]]

        contrastList.append('contrastList_' + task)
        contrastList[i] = [cont01, cont02, cont03, cont04, cont05]
        
    return contrastList

### report_activation

In [ ]:
def get_enconew_info():
    enconewDirlist = [x for x in os.listdir(opj(subDatasinkDir, '1stLevel')) if x.startswith('enconew')]
    return enconewDirlist

In [ ]:
def report_activation(task):
    """
    print some nice figures to aggregate in a report
    will print automagically:
    - the design matrix, 
    - the anatomy in NMI space, 
    - a glass brain with the activation 
        
    """
    
    #try:
    #    fileID = subID + '_task-' + task.split('_')[0] + '_bold'
    #    taskOrig = task.split('_')[0]
    #except:
    #    fileID = subID + '_task-' + task + '_bold'
    
    fileID = subID + '_task-' + task + '_bold'
    
    numOutliers = file_len(opj(subDatasinkDir, 'preproc', 'art.r' + fileID + '_outliers.txt'))
    print('The number of outliers is: %s' % numOutliers)
    print('For a nicer figure, define the aspect ratio for the plot of the Design Matrix')
    print('[default = 1 (square), lower is vertical rectangle, higher is horizontal rectangle]')
    aspectInput = input('Aspect ratio: (number between 0.5 and 2.5) default [1.0]: ')
    if aspectInput == '':
        aspectRatio = 1.0
    else:
        try:
            aspectRatio = float(aspectInput)
            if 0.5 <= aspectRatio <= 2.5:
                pass
            else:
                print('Number out of range, taking default [1.0].')
        except:
            aspectRatio = 1.0
            print('Not a number, taking default [1.0].')

    # Using scipy's loadmat function we can access SPM.mat
    spmmat = loadmat(opj(subDatasinkDir, '1stLevel', task, 'SPM.mat'),
                     struct_as_record=False)

    designMatrix = spmmat['SPM'][0][0].xX[0][0].X
    names = [i[0] for i in spmmat['SPM'][0][0].xX[0][0].name[0]]

    normedDesign = designMatrix / np.abs(designMatrix).max(axis=0)

    fig, ax = plt.subplots(figsize=(int(round(aspectRatio * 8)), 8))
    plt.imshow(normedDesign, aspect='auto', cmap='gray', interpolation='none')
    plt.suptitle('Design matrix for ' + task.upper())
    ax.set_ylabel('Volume id')
    ax.set_xticks(np.arange(len(names)))
    ax.set_xticklabels(names, rotation=90)
    plt.show()
    #fig.savefig(opj(subDatasinkDir, 'figures', 'DesignMatrix_' + taskOrig + '.png'));
    fig.savefig(opj(subDatasinkDir, 'figures', 'DesignMatrix_' + task + '.png'));
    
    anatImg = opj(subDatasinkDir, 'preproc', 'w' + subID +'_T1w.nii')
    #epiImg4D = opj(subDatasinkDir, 'preproc', 'wdetrend-' + taskOrig + '.nii')
    epiImg4D = opj(subDatasinkDir, 'preproc', 'wdetrend-' + task + '.nii')
    epiImg = nli.index_img(epiImg4D, 1)
    
    if 'enco' in task:
        contrast = 'con_0004.nii'
    else:
        contrast = 'con_0005.nii'
    
    if task == 'enconew' and len(enconewDirlist) > 1:
        for enTask in enconewDirlist:
            conImg = opj(subDatasinkDir, '1stLevel', enTask, contrast)
            plot_glass_brain(conImg, threshold=0.5, title = enTask.upper() + ': Activation on GlassBrain')
            posConImg = math_img('img1*img2', img1 = conImg, img2 = math_img('img>0', img = conImg))
            plot_stat_map(posConImg, threshold=0.25, colorbar=False, title = enTask.upper() + ' on standard brain')
            # Save the images
            plot_glass_brain(conImg, threshold=0.5, title = enTask.upper() + ': Activation on GlassBrain', output_file = opj(subDatasinkDir, 'figures', 'GlassBrain_' + enTask + '.png'))
            plot_stat_map(posConImg, threshold=0.25, colorbar=False, title = enTask.upper() + ' on standard brain', output_file = opj(subDatasinkDir, 'figures',enTask.upper() + '_on_standard_brain.png'))
    else:
        conImg = opj(subDatasinkDir, '1stLevel', task, contrast)
        plot_glass_brain(conImg, threshold=0.5, title = task.upper() + ': Activation on GlassBrain')
        posConImg = math_img('img1*img2', img1 = conImg, img2 = math_img('img>0', img = conImg))
        plot_stat_map(posConImg, threshold=0.25, colorbar=False, title = task.upper() + ' on standard brain')
        # Save the images
        plot_glass_brain(conImg, threshold=0.5, title = task.upper() + ': Activation on GlassBrain', output_file = opj(subDatasinkDir, 'figures', 'GlassBrain_' + task + '.png'))
        plot_stat_map(posConImg, threshold=0.25, colorbar=False, title = task.upper() + ' on standard brain', output_file = opj(subDatasinkDir, 'figures', task.upper() + '_on_standard_brain.png'))
    
    #if 'enco' not in task:
    #    conImg = opj(subDatasinkDir, '1stLevel', task, 'con_0005.nii')
    #else:
    #    conImg = opj(subDatasinkDir, '1stLevel', task, 'con_0004.nii')

    #plot_glass_brain(conImg, threshold=0.5, title = task.upper() + ': Activation on GlassBrain')
    
    #posConImg = math_img('img1*img2', img1 = conImg,
    #                                     img2 = math_img('img>0', img = conImg))
    
    #plot_stat_map(posConImg, threshold=0.25, colorbar=False, title = task.upper() + ' on standard brain')
    
    # Save the images
    #plot_glass_brain(conImg, threshold=0.5, title = task.upper() + ': Activation on GlassBrain', output_file = opj(subDatasinkDir, 'figures', 'GlassBrain_' + task + '.png'))
    #plot_stat_map(posConImg, threshold=0.25, colorbar=False, title = task.upper() + ' on standard brain', output_file = opj(subDatasinkDir, 'figures', task.upper() + '_on_standard_brain.png'))

### plotparam_spmcontrast

In [ ]:
def plotparam_spmcontrast(task):
    """
    Print a page of SPM overlayed on anatomy or EPI image
    
    returns plotParam, a dictionary with plotting parameters
        
    """
        
    # dictionary of paramaters for plotting
    plotParam = {}
    
    #try:
    #    taskNew = task.split('_')[0]
    #except:
    #    taskNew = task
        
    taskNew = task
    
    # Show a list of contrasts
    print('')
    print('List of spm contrasts for task ' + taskNew.upper())
    for i, item in enumerate(contrastList[firstlevelList.index(taskNew)]):
        if item[1][-1] == 'T':
            print('  ' + str(i+1) + '. spmT_000' + str(i+1) + '.nii = ' + str(item[:2]) + str(item[-1]))
        else:
            print('  ' + str(i+1) + '. spmF_000' + str(i+1) + '.nii = ' + str(item[:2]))
            
    spmNum = int(input('Select a contrast by choosing the corresponding number: '))
    print('')
    
    if contrastList[firstlevelList.index(taskNew)][spmNum-1][1] == 'T':
        plotParam['c2p'] = 'spmT_000' + str(spmNum)
        thrshld = input('Lower T cut-off value: [default = 3.5]')
        if thrshld == '':
            plotParam['thrshld'] = 3.5
        else:
            try:
                plotParam['thrshld'] = float(thrshld)
            except:
                print('Wrong entry, taking 3.5 for T-threshold.')
        maxVal = input('Highest T value: [default = None]')
        if maxVal == '':
            plotParam['maxv'] = None
        else:
            try:
                plotParam['maxv'] = int(maxVal)
            except:
                print('Wrong entry, taking "None" for maxv.')
    else:
        plotParam['c2p'] = 'spmF_000' + str(spmNum)
        thrshld = input('Lower F cut-off value: [default = 20]')
        if thrshld == '':
            plotParam['thrshld'] = 20
        else:
            try:
                plotParam['thrshld'] = float(thrshld)
            except:
                print('Wrong entry, taking 20 for F-threshold.')
        maxVal = input('Highest T value: [default = None]')
        if maxVal == '':
            plotParam['maxv'] = 200
        else:
            try:
                plotParam['maxv'] = int(maxVal)
            except:
                print('Wrong entry, taking 200 for maxv.')
    
    anatImg = opj(subDatasinkDir, 'preproc', 'w' + subID +'_T1w.nii')
    epiImg4D = opj(subDatasinkDir, 'preproc', 'wdetrend-' + taskNew + '.nii')
    epiImg = nli.index_img(epiImg4D, 1)
       
    # inquire which direction to plot, sag, cor or ax
    print("")
    plotDirection = input('Select a plotting direction (sagittal = x, coronal = y, axial = z): ')
    if plotDirection.lower() == 'x' or plotDirection.lower().startswith('sag'):
        plotParam['first'] = -60
        plotParam['last'] = 60
        plotParam['increment'] = int(math.ceil((plotParam['last']-plotParam['first'])/float(25)))
        plotParam['title'] = task.upper() + ' L>R'
        print('selected sagittal')
        plotParam['displ_mod'] = 'x'
    elif plotDirection.lower() == 'y' or plotDirection.lower().startswith('cor'):
        plotParam['first'] = -80
        plotParam['last'] = 30
        plotParam['increment'] = int(math.ceil((plotParam['last']-plotParam['first'])/float(25)))
        plotParam['title'] = task.upper()
        print('selected coronal')
        plotParam['displ_mod'] = 'y'
    elif plotDirection.lower() == 'z' or plotDirection.lower().startswith('ax'):
        plotParam['first'] = -50
        plotParam['last'] = 75
        plotParam['increment'] = int(math.ceil((plotParam['last']-plotParam['first'])/float(25)))
        plotParam['title'] = task.upper()
        print('selected axial')
        plotParam['displ_mod'] = 'z'
    
    print(opj(subDatasinkDir, '1stLevel', task, plotParam['c2p'] + '.nii'))
    
    #plotParam['image'] = anatimg
    backgroundIma = input('Background anatomical or EPI [default = anatomical]?: ')

    if backgroundIma.lower().startswith('e'):
        plotParam['image'] = epiImg
        plotParam['dimming'] = 0
    else:
        plotParam['image'] = anatImg
        plotParam['dimming'] = -1
    
    return plotParam

### aggregate_images

In [ ]:
def aggregate_images(imageList, plotParam):
    """
    Function for stacking images vertically and saving in an aggregated image file
    
    saves aggr_image.<extention> in the dir of the images
    
    returns the name of the new image including full path
    
    """
   
    plotContrast = plotParam['c2p']
    
    extention = imageList[0].split('.')[-1]

    images = list(map(pimage.open, imageList))
    widths, heights = list(zip(*(i.size for i in images)))
    
    maxWidth = max(widths)
    totalHeight = sum(heights)
    
    newImg = pimage.new('RGB', (maxWidth, totalHeight))
    
    yOffset = 0
    for im in images:
        newImg.paste(im, (0, yOffset))
        yOffset += im.size[1]
        
    newImgName = imageList[0].split('_1.' + extention)[0] + '_' + plotContrast + '.' + extention
    
    newImg.save(opj(figDir, newImgName))
    
    print('New Image %s saved.' % newImgName)
    print('Image size = %s' % str(newImg.size))
    
    return newImgName

### plot_fMRI_page

In [ ]:
def plot_fMRI_page(plotParam, task, print2screen='y'):
    """
    Inside function to plot to screen or to file
    
    print2screen is 'y' or 'n' in wich case safe to file
    
    returns outfileList, a list with the paths to the created images
    
    """
    
    #plotParam['title'] = task.upper()
    
    try:
        if 't1w' in plotParam['image'].lower():
            bckgrnd = 'anat'
    except:
        bckgrnd = 'epi'
        
    if plotParam['displ_mod'] == 'x': modPlt = 'sag'
    if plotParam['displ_mod'] == 'y': modPlt = 'cor'
    if plotParam['displ_mod'] == 'z': modPlt = 'tra'
        
    outfileList = []
     
    index = 1

    for n in range(plotParam['first'], plotParam['last'], (5*plotParam['increment'])):    

        coordinates = []

        for i in range(n, (n + (5*plotParam['increment'])), plotParam['increment']):
            coordinates.append(i)

        coord = tuple(coordinates)
        
        # threshold the contrast to only see the positive values
        
        img2Thrhld = opj(subDatasinkDir, '1stLevel', task, plotParam['c2p'] + '.nii')
        posSPMT = math_img('img1*img2', img1 = img2Thrhld,
                                         img2 = math_img('img>0', img = img2Thrhld))

        if print2screen.lower().startswith('y'):
            plot_stat_map(posSPMT, 
                            title=plotParam['title'] + ' : T>' + str(plotParam['thrshld']), vmax = plotParam['maxv'], bg_img=plotParam['image'], 
                            threshold=plotParam['thrshld'], display_mode=plotParam['displ_mod'], cut_coords=coord, colorbar=False, 
                            dim=plotParam['dimming'], output_file = None);
            plt.show()
        elif print2screen.lower().startswith('n'):
            outFile = (opj(subDatasinkDir, 'figures', 'overlay_' + bckgrnd + '_' + task.upper() + '_' + modPlt + '_' + str(index) + '.png'))
            print('File will be saved to %s' % outFile)
            plot_stat_map(posSPMT, 
                            title=plotParam['title'] + ' : T>' + str(plotParam['thrshld']), vmax = plotParam['maxv'], bg_img=plotParam['image'], colorbar=False, 
                            threshold=plotParam['thrshld'], display_mode=plotParam['displ_mod'], cut_coords=coord, 
                            dim=plotParam['dimming'], output_file = outFile);
            plt.show()
            outfileList.append(outFile)

        index = index + 1

        n = n + (5*plotParam['increment'])
                
    return outfileList

### show_spmresult

In [ ]:
def show_spmresult(task):
    """
    function to print the spm on background to the output
    with the possibility to change parameters
    
    input: task, should be in de firstlevelList
    
    """
    
    print('Generating output for %s' % task)
    #to continue

In [ ]:
def human_size(bytes, units=[' bytes','KB','MB','GB','TB', 'PB', 'EB']):
    """ 
    Returns a human readable string reprentation of bytes by using >>, a bitwise operator 
    
    """
    
    return str(bytes) + units[0] if bytes < 1024 else human_size(bytes>>10, units[1:])

### LI curves

In [ ]:
def selectROIs():
    """
    Procedure to select brain regions from AAL
    
    returns ROIList
    
    """
    
    ROIList = []
    
    print(brainRegions)
    region = input('\nType a brain region to add to ROI: <return to stop>\n')

    while region != '':
        if region in brainRegions and region not in ROIList:
            ROIList.append(region)
            region = input('\nType a brain region to add to ROI: <return to stop>\n')
        else:
            print('\nOeps, typo or already selected. Try again!\n')
            region = input('\nType a brain region to add to ROI: <return to stop>\n')
        
    print(ROIList)
            
    return ROIList

In [ ]:
def createROIs(ROIList):
    """
    Function to create composite left and right ROIs from the regions in ROIList

    returns leftROI and rightROI
    
    """
    
    print('\nCreating the composite ROIs from this list: %s\n' %str(ROIList))
    AAL = np.array(aalImg.dataobj)
    leftROI = np.zeros(AAL.shape)
    rightROI = np.zeros(AAL.shape)
    
    for region in ROIList:
        leftROIs = aalROIs.loc[(aalROIs.Name.str.contains(region)) & (aalROIs.Name.str.contains('_L'))]
        rightROIs = aalROIs.loc[(aalROIs.Name.str.contains(region)) & (aalROIs.Name.str.contains('_R'))]

        leftROIList = list(leftROIs.loc[:,'Number'])
        for element in leftROIList:
            leftROI += (AAL == element).astype(int)

        rightROIList = list(rightROIs.loc[:,'Number'])
        for element in rightROIList:
            rightROI += (AAL == element).astype(int)
    
    return leftROI, rightROI

In [ ]:
def LI_curves(leftROI, rightROI, spmT, fN = 0.1):
    """
    Procedure to create Laterality Index (LI) array's to be plotted
    LeftROI and rightROI are the ROI's for which the LI-curves have to be calculated
    spmT is the nifti file of the contrast
    fN is the finese for calculating the LI series (best leave unchanged)
    
    returns aIndexDict, with 2 arrays with keys:
        aIndexNum (sum of voxels above theshold)
        aIndexWei (sum of of voxels * T-value above threshold)
        
    """
    # resample the spmT image to the AAL template (loaded at start-up)
    resampled_spmT = resample_to_img(spmT, aalImg)
    
    # import image as numpy array (3D)
    resArr = np.array(resampled_spmT.dataobj)
    
    # calculate the masked spmT arrays
    leftROI_masked_spmT = leftROI * resArr
    rightROI_masked_spmT = rightROI * resArr
        
    # define a dict with 2 arrays
    aIndexDict = {}
    aIndexDict['aIndexNum'] = []
    aIndexDict['aIndexWei'] = []
    
    # define the max T-value of the spmT array
    Tmax = max(np.max(leftROI_masked_spmT), np.max(rightROI_masked_spmT))//1.5 # not to have to many 1.0's in LI series
    
    # create the LI series
    for tH in range(0, int((Tmax // 1)/fN)):
        leftROI_thrshld = leftROI_masked_spmT[leftROI_masked_spmT>(tH*fN)]
        rightROI_thrshld = rightROI_masked_spmT[rightROI_masked_spmT>(tH*fN)]

        leftNum = leftROI_thrshld.shape[0]
        rightNum = rightROI_thrshld.shape[0]

        leftW = sum(leftROI_thrshld)
        rightW = sum(rightROI_thrshld)


        aIndexNum = (leftNum - rightNum) / (leftNum + rightNum)
        aIndexW = (leftW - rightW) / (leftW + rightW)


        aIndexDict['aIndexNum'].append(aIndexNum)
        aIndexDict['aIndexWei'].append(aIndexW)
        
    return aIndexDict

### Fig Gen

In [ ]:
def fMRI_fig(task):
    """
    Function to generate an SPM with correct T cut-off
    Select T value, ax, cor or sag and coordinate limits
    
    """
        
    plotParam = plotparam_spmcontrast(task)
    outfileList = plot_fMRI_page(plotParam, task, print2screen='y')        
    
    answer1 = input('\nChange the threshold? y/[n]')
    
    while answer1.upper().startswith('Y'):
        newT = input('Value = ')
        plotParam['thrshld'] = float(newT)
        outfileList = plot_fMRI_page(plotParam, task, print2screen='y')
        answer1 = input('\nChange the threshold? y/[n]')
    
    answer2 = input('\nChange the range of the plot? y/[n]')
    
    while answer2.upper().startswith('Y'):
        first = input('New value for first slice (now %s): ' % plotParam['first'])
        plotParam['first'] = int(first)
        last = input('New value for last slice (now %s): ' % plotParam['last'])
        plotParam['last'] = int(last)
        increment = input('New value for increment (now %s): ' % plotParam['increment'])
        plotParam['increment'] = int(increment)        
        outfileList = plot_fMRI_page(plotParam, task, print2screen='y')
        answer2 = input('\nChange the range of the plot? y/[n]')
        
    answer3 = input('Do you want to save this result: y/[n]')
    
    if answer3.upper().startswith('Y'):
        outfileList = plot_fMRI_page(plotParam, task, print2screen='n')
        newImgName = aggregate_images(outfileList, plotParam)
        print(newImgName)

        # delete the individual overlay files
        for ff in outfileList:
            os.remove(ff)

        try:
            if 'T1w' in plotParam['image']:
                title = 'Overlay of T1 anatomy and ' + plotParam['c2p'] + ' for ' + task.upper()
                print('')
                print('')
                printmd(title, color = 'blue')    
        except:
            title = 'Overlay of EPI and ' + plotParam['c2p'] + ' for ' + task.upper()
            print('')
            print('')
            printmd(title, color = 'blue')

        im = pimage.open(newImgName)
        plt.figure(figsize = (24,16))
        plt.axis('off')
        plt.imshow(im)

    else:
        print('Figure not saved!')
    
    return plotParam, outfileList

In [ ]:
def LI_fig(task):
    """
    Function to generate activation laterality curves
    
    """
    
    spmT = opj(subDatasinkDir, '1stLevel', task, plotParam['c2p'] + '.nii')
    tHold = plotParam['thrshld']
    
    answer1 = input('\nLI Curves? y/[n]')
    
    while answer1.upper().startswith('Y'):
        if task.startswith('enco'):
            #ROIList = ['Hippocampus', 'ParaHippocampal', 'Fusiform', 'Amygdala']
            ROIList = selectROIs()
        else:
            ROIList = selectROIs() 
        
        leftROI, rightROI = createROIs(ROIList)
        aIndexDict = LI_curves(leftROI, rightROI, spmT, fN=0.1)
        
        region = ''.join(ROIList)
        figName = opj(subDatasinkDir, 'figures', 'LI_' + task.upper() + '_' + region + '.png')
        
        fN = 0.1
        numTrace = np.asarray(aIndexDict['aIndexNum'])
        weiTrace = np.asarray(aIndexDict['aIndexWei'])

        X = [x*fN for x in range(0, len(weiTrace))]
        
        fig = plt.figure()
        fig.set_size_inches(7, 5)        
        
        plt.plot(X, weiTrace, label = 'Weighted')
        plt.plot(X, numTrace, label = 'Numbers')
        plt.legend(loc='upper left')
        xlabel('T-value')
        ylabel('Assymmetry Index')
        plt.title(task.upper()+': '+str(ROIList) , fontdict=None, loc='center', pad=None)
        plt.axvline(x=tHold)
        #plt.show()
        plt.savefig(figName, format='png', dpi=300)
        plt.show()
        
        answer2 = input('Save figure? y/[n]')

        if answer2.upper().startswith('N') or answer2 == '': 
            os.remove(figName)
            print('LI figure deleted.')
        
        answer1 = input('LI Curves? y/[n]')

In [ ]:
def svgtopng(task):
    """
    Convert the error volume svg to png format
    
    """
    
    svgFile = opj(subDatasinkDir, 'preproc', 'plot.r' + subID + '_task-' + task + '_bold.svg')
    pngFile = opj(subDatasinkDir, 'preproc', 'plot.r' + subID + '_task-' + task + '_bold.png')
    
    svg2png(url=svgFile, write_to=pngFile)
    
    print('converted %s to %s' % (svgFile, pngFile))

In [ ]:
def show_subjects():
    """
    function to display a tabel of the subjects in /data/Patients/SubjectInfoFiles/subjects.tsv
    
    """

    print(' #   Subject                                  Adrema          Accession  Date Exam  Referral                                           fMRI tasks                                                       DTI     F-map     Figs    Report')
    print('-'*220)
    with open(opj(iDir, 'subjects.tsv'), 'rt') as tsvFile:
        reader = csv.reader(tsvFile, delimiter='\t')
        #sortedLines = sorted(reader, key=operator.itemgetter(3))
        idx = 0
        for row in reader:
            idx = idx + 1
            row = [idx] + row
            print(' {:<3} {:<40} {:<15} {:<10} {:<10} {:<50} {:<65} {:<8} {:<8} {:<8} {:<8}'.format(*row))

In [ ]:
def update_subjects():
    """
    Function to update certain parameters in /data/Patients/SubjectInfoFiles/subjects.tsv
    
    """
    
    show_subjects()
    
    with open(opj(wDir, 'SubjectInfoFiles', 'subjects.tsv'), 'rt') as tsv_file:
        reader = csv.reader(tsv_file, delimiter='\t')
        lines = list(reader)

    rows = len(lines) 
    
    try:
        answer = int(input('\nWelke lijn wilt u aanpassen?:'))-1
        print(lines[answer])
        
        param2Change = {'Referral':4, 'Figs':8, 'Report':9}

        if answer in range(rows) and answer != 0:
            
            item = input('\nWelke waarde wil je veranderen? (kies uit: %s): ' % (', '.join(str(i) for i in list(param2Change.keys()))))
            if item in param2Change.keys():
                position = param2Change[item]
                print("\nHuidige waarde variable '%s' = %s" % (item, lines[answer][position]))
                newvalue = input("\nNieuwe waarde voor variabele '%s' = " % item)
                lines[answer][position] = newvalue
                
                print(lines[answer])
                
                with open(opj(wDir, 'SubjectInfoFiles', 'newsubjects.tsv'), 'w') as tsv_file:
                    writer = csv.writer(tsv_file, delimiter='\t')
                    writer.writerows(lines)
                
                shutil.copy(opj(wDir, 'SubjectInfoFiles', 'subjects.tsv'), opj(wDir, 'SubjectInfoFiles', 'subjects.tsv.backup'))
                shutil.copy(opj(wDir, 'SubjectInfoFiles', 'newsubjects.tsv'), opj(wDir, 'SubjectInfoFiles', 'subjects.tsv'))
                os.remove(opj(wDir, 'SubjectInfoFiles', 'newsubjects.tsv'))
                
                clear_output()
                
                print('Dit is the aangepaste subjects.tsv file:\n')
                show_subjects()
                
            else:
                print('Deze parameter bestaat niet!')

        else:
            print('Deze rij bestaat niet (of is hoofding)!')

    except:
        print('Fout! Duid een lijn aan met een nummer!')

# Start generating figures

In [ ]:
patientStatList = subjects4report()

In [ ]:
subjectsWOR = subjectsWithoutReport()

In [ ]:
subID, firstlevelList = select_subject4figures(subjectsWOR)

In [ ]:
# define some useful variables for further processing
subDir = opj(wDir, subID)
outputDir = opj(oDir, subID)
subDatasinkDir = opj(oDir, 'results_' + subID + '-N')
figDir = opj(subDatasinkDir, 'figures')

In [ ]:
# create a directory for the figures
dir2Create = subDatasinkDir + '/figures'
if os.path.isdir(dir2Create):
    print('Directory %s exists! Nothing done.' % dir2Create)
else:
    os.makedirs(dir2Create)
    print('Directory %s created!' % dir2Create)

In [ ]:
contrastList = contrastList_gen()

In [ ]:
firstlevelList# = ['wgen', 'read', 'encoold', 'enconew']

In [ ]:
# Visualize the realignment plots and save copies to disc under the ../results sections per task

for task in firstlevelList:
    fileID = subID + '_task-' + task + '_bold'
    par = np.loadtxt(opj(subDatasinkDir, 'preproc', 'rp_' + fileID + '.txt'))
    fig, axes = plt.subplots(2, 1, figsize=(15, 5))
    plt.suptitle('Realignment plot of ' + task.upper())
    axes[0].set_ylabel('rotation (radians)')
    axes[0].plot(par[0:, :3])
    axes[1].plot(par[0:, 3:])
    axes[1].set_xlabel('time (scans)')
    axes[1].set_ylabel('translation (mm)');
    plt.savefig('realignmentplot.png')
    copy2Dir = subDatasinkDir + '/figures'
    shutil.copy('realignmentplot.png', copy2Dir + '/realignmentplot_' + task + '.png')

In [ ]:
# plot the artefact detection timelines
for task in firstlevelList:
    plot_art(task)

In [ ]:
for task in firstlevelList:
    fileID = subID + '_task-' + task + '_bold'
    numOutliers = file_len(opj(subDatasinkDir, 'preproc', 'art.r' + fileID + '_outliers.txt'))
    print('Number of artefacted volumes in ' + task + ' = ' + str(numOutliers))

In [ ]:
voxel = plot_detrended_voxel()

## Select task

In [ ]:
task = select_task()

In [ ]:
report_activation(task)

In [ ]:
try:
    svgtopng(task)
except:
    print('No error volumes svg file for %s' % task)

In [ ]:
plotParam, outfileList = fMRI_fig(task)

In [ ]:
LI_fig(task)

## Generate SPM directory

In [ ]:
# Put all files necessary for SPM results into a directory SPM under <oDir>/results_sub_ID/figures
SPMdir = opj(subDatasinkDir, 'figures', 'SPM')

if os.path.isdir(SPMdir):
    print('Directory %s exists! Nothing done.' % SPMdir)
else:
    os.makedirs(SPMdir)
    print('Directory %s created!' % SPMdir)

In [ ]:
# copy the wdetrend en wanat files
for f in os.listdir(opj(subDatasinkDir, 'preproc')):
    if f.startswith('w'):
        shutil.copy(opj(subDatasinkDir, 'preproc', f), opj(SPMdir, f))

# copy the SPM12 files
for task in firstlevelList:
    #print(task)
    # copy the task SPM.mat files and rename
    shutil.copy(opj(subDatasinkDir, '1stLevel', task, 'SPM.mat'), opj(SPMdir, 'SPM_' + task + '.mat'))

    # copy the spm_#####.nii files corresponding to the saved figures and rename
    for f in os.listdir(opj(subDatasinkDir, 'figures')):
        if task.upper() in f and f.startswith('overlay_'):
            nbr = f.split('.')[0].split('_')[-1]
            cntrst = f.split('.')[0].split('_')[-2]
            shutil.copy(opj(subDatasinkDir, '1stLevel', task, cntrst + '_' + nbr + '.nii'), opj(SPMdir, cntrst + '_' + nbr + '_' + task + '.nii'))

In [ ]:
print('Files in SPM dir: ')
for i, f in enumerate(os.listdir(SPMdir)):
    fbytes = os.path.getsize(opj(SPMdir, f))
    fsize = human_size(fbytes, units=[' bytes','KB','MB','GB','TB', 'PB', 'EB'])
    print('%i. %s - %s' % (i+1, f, fsize))

## Update subjects.tsv

In [ ]:
update_subjects()